<a href="https://colab.research.google.com/github/StefaniaGutu/OffensiveLanguageClassification/blob/main/NLP_Project_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
import string
import spacy
import pickle
from os.path import exists
from gensim.models import Word2Vec as w2v
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import classification_report
import math

Citire dataset

In [24]:
df = pd.read_csv('sample_data/labeled_data.csv')
dataset_labels = df["class"]
dataset_tweets = df["tweet"]
print(dataset_labels[:10])

0    2
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: class, dtype: int64


Preprocesare

In [25]:
def preprocess(dataset):
    # eliminare @user
    tweets_no_user = [re.sub(r'@[\w]*', '', tweet) for tweet in dataset_tweets]

    # eliminare URL
    tweets_no_links = [re.sub(r'http\S+', '', tweet) for tweet in tweets_no_user]

    # eliminare hashtags
    tweets_no_hashtags = [re.sub(r'#[a-zA-Z0-9_]+','', tweet) for tweet in tweets_no_links]

    # eliminare &_;
    tweets_no_apersand = [re.sub(r'&[a-zA-Z0-9_]+','', tweet) for tweet in tweets_no_hashtags]

    # eliminare caractere speciale
    tweets_no_punct = [tweet.translate(str.maketrans('', '', string.punctuation)) for tweet in tweets_no_apersand]

    # eliminare cifre
    tweets_no_digits = [re.sub(r'\d+', '', tweet) for tweet in tweets_no_punct]

    # transformare caractere in litere mici
    tweets_lower = [tweet.lower() for tweet in tweets_no_digits]

    # inlocuire secvente repetitive cu aceeasi litera cu 2 doua repetitii a acesteia
    # e.g. 'wooooow' -> 'woow'
    tweets_no_repetitions = [re.sub(r'(.)\1+', r'\1\1', tweet) for tweet in tweets_lower]

    # inlocuire multiple caractere albe cu un singur caracter alb
    tweets_no_spaces = [re.sub(r'\s+', ' ', tweet) for tweet in tweets_no_repetitions]

    # lemmatizare
    nlp = spacy.load('en_core_web_sm')
    all_words_lemma = [[word.lemma_ for word in nlp(sent)] for sent in tweets_no_spaces]

    # eliminare stopwords
    stop_words_spacy = nlp.Defaults.stop_words
    all_words_without_stops = [[word for word in sent if word not in stop_words_spacy] for sent in all_words_lemma]

    # eliminare cuvinte cu mai putin de doua caractere
    words_no_shortWords = [[word for word in tweet if len(word)>2] for tweet in all_words_without_stops]
    tweets_no_shortWords = [word for word in words_no_shortWords]

    return tweets_no_shortWords

In [26]:
preprocessed_dataset = preprocess(dataset_tweets)
preprocessed_dataset[:15]

[['woman', 'complain', 'clean', 'house', 'man', 'trash'],
 ['boy', 'dat', 'coldtyga', 'dwn', 'bad', 'cuffin', 'dat', 'hoe', 'place'],
 ['dawg', 'fuck', 'bitch', 'start', 'cry', 'confuse', 'shit'],
 ['look', 'like', 'tranny'],
 ['shit', 'hear', 'true', 'faker', 'bitch', 'tell'],
 ['shit', 'blow', 'meclaim', 'faithful', 'somebody', 'fuck', 'hoe'],
 ['sit', 'hate', 'bitch', 'shit'],
 ['cause', 'tired', 'big', 'bitch', 'come', 'skinny', 'girl'],
 ['bitch'],
 ['hobby', 'include', 'fight', 'mariam', 'bitch'],
 ['keek', 'bitch', 'curve', 'lol', 'walk', 'conversation', 'like', 'smh'],
 ['murda', 'gang', 'bitch', 'gang', 'land'],
 ['hoe', 'smoke', 'loser', 'yea'],
 ['bad', 'bitch', 'thing', 'like'],
 ['bitch']]

In [27]:
print(dataset_labels[:10])
# salvare date preprocesate
pck = open("./pickles/preprocessed_dataset.pck", "wb")
pickle.dump({
    'preprocessed_dataset': preprocessed_dataset,
    'dataset_labels': dataset_labels
}, pck)

0    2
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: class, dtype: int64


In [28]:
# incarcare date preprocesate
dataset_tweets = []
dataset_labels = []

pck = open("./pickles/preprocessed_dataset.pck", "rb")
result = pickle.load(pck)
dataset_tweets = result['preprocessed_dataset']
dataset_labels = result['dataset_labels']

Extragere features

In [29]:
# Label-uri originale
# 0 - hate speech 
# 1 - offensive language 
# 2 - neither

# Transformare New Labels
# 0 - hate speech & offensive language 
# 1 - normal speech

new_dataset_labels = []
for i in dataset_labels:
  if i == 1:
    new_dataset_labels.append(0)
  elif i == 2:
    new_dataset_labels.append(1)
  else:
    new_dataset_labels.append(i)

hate_speech = [data for data in new_dataset_labels if data == 0]
print('Hate speech tweets: ', len(hate_speech))
print('Normal speech tweets: ', len(new_dataset_labels) - len(hate_speech))

Hate speech tweets:  20620
Normal speech tweets:  4163


Vectorizare tweets

In [30]:
w = w2v(
    dataset_tweets,
    min_count=3,
    sg=1,  # folosire skip-gram
    window=7
)

emb_df = (
    pd.DataFrame(
        [w.wv.get_vector(str(n)) for n in w.wv.key_to_index],
        index=w.wv.key_to_index
    )
)

print("Snippet of vectorized dataset:")
emb_df.head()

Snippet of vectorized dataset:


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
bitch,-0.186601,0.303706,0.226702,0.129422,0.234953,-0.269451,0.315455,0.545220,-0.492730,-0.088232,...,0.084412,0.037535,0.270657,-0.000025,0.606244,0.149338,-0.102350,-0.044696,-0.047560,0.164906
hoe,-0.252885,0.218358,0.141698,0.125353,0.318981,-0.437701,0.179263,0.461769,-0.420013,-0.242943,...,-0.003474,0.179359,0.224140,0.031420,0.534363,0.088017,0.191425,-0.144000,-0.277114,0.109153
like,-0.045649,0.008154,-0.069060,-0.140555,0.215516,-0.273308,0.025383,0.493558,-0.327128,0.067989,...,-0.044141,0.313047,0.356153,-0.151202,0.438989,0.344465,0.227263,0.030640,0.169600,-0.015206
pussy,-0.328523,0.158818,0.004122,0.088795,0.285280,-0.328634,0.074066,0.376174,-0.186576,-0.320660,...,0.014798,0.189353,-0.073312,0.025972,0.652564,0.058487,0.185801,-0.192184,-0.224714,0.018949
fuck,0.017596,0.045075,-0.091222,-0.015126,0.060066,-0.261692,0.106155,0.439016,-0.286701,-0.056454,...,0.272354,0.223339,0.041651,0.143605,0.451436,0.165984,0.042076,-0.009220,-0.012717,0.008747


In [31]:
vectorized_sentences = [] 
vectorized_labels = []

for i, sentence in enumerate(dataset_tweets):
    sentence_vectors = []
    for word in sentence:
        try:
            word_vector = w.wv.get_vector(word)
            word_vector = word_vector[~np.isnan(word_vector)]
            sentence_vectors.append(np.mean(word_vector))
        except KeyError:
            # If a word is not in the vocabulary, ignore it
            pass
    try:
      vectorized_sentence = sentence_vectors
      vectorized_sentences.append(vectorized_sentence)
      vectorized_labels.append(new_dataset_labels[i])
    except ZeroDivisionError:
      pass

print("Word vectorization for every sentence:")
print(vectorized_sentences[:5])
print("Labels for the above sentences: ", vectorized_labels[:5])

Word vectorization for every sentence:
[[0.02035542, 0.013291196, 0.014167927, 0.011339133, 0.019492727, -0.007226741], [0.007143587, 0.01004994, 0.019328563, 0.011600499, 0.01004994, 0.013193748, 0.013349113], [0.014054215, 0.015724724, 0.0123932995, 0.012694121, 0.01706051, 0.012510509, 0.004316042], [0.015409739, 0.0196084, 0.011368518], [0.004316042, 0.013159556, 0.014326045, 0.008302736, 0.0123932995, 0.018008757]]
Labels for the above sentences:  [1, 0, 0, 0, 0]


Adaugare padding

In [32]:
# padding every tweet to the longest sentence
padded_tweets = pad_sequence([torch.tensor(tweet) for tweet in vectorized_sentences], batch_first=True, padding_value=0)

Separarea datelor in date de antrenare, testare si validare

In [33]:
X_train, X_val_test, y_train, y_val_test = train_test_split(padded_tweets, vectorized_labels, test_size=0.30, random_state=42)
X_test, X_validation, y_test, y_validation = train_test_split(X_val_test, y_val_test, test_size=0.50, random_state=42)

print(len(X_train))
print(len(X_test))
print(len(X_validation))

17348
3717
3718


Model

In [34]:
# Convertire date in tensori PyTorch

X_train_tensor = torch.tensor(X_train)
X_val_tensor = torch.tensor(X_validation)
X_test_tensor = torch.tensor(X_test)
y_train_tensor = torch.tensor(y_train)
y_val_tensor = torch.tensor(y_validation)
y_test_tensor = torch.tensor(y_test)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
val_data = TensorDataset(X_val_tensor, y_val_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

<ipython-input-34-5912342b6509>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(X_train)
<ipython-input-34-5912342b6509>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_tensor = torch.tensor(X_validation)
<ipython-input-34-5912342b6509>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test)


In [35]:
# dataloaders
batch_size = 50  
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [36]:
# definire model RNN
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.shape[0], 1, -1)  # reshape la (batch_size, sequence_length, input_size)
        out, _ = self.rnn(x)
        out = out[:, -1, :]  # get the last output of the sequence 
        out = self.fc(out)
        return out

model = RNN(len(X_train[0]), 32, 2)  # clasificare binara (hate_speech & offensive_language vs. normal_speech)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

Antrenare si validare model

In [37]:
epochs_no = 100
for epoch in range(epochs_no):  # antrenare model in epoci 
    # antrenare
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

    # validare
    model.eval()
    
    val_losses = []
    for inputs, labels in val_loader:
        with torch.no_grad():
            val_outputs = model(inputs.float())
            val_loss = criterion(val_outputs, labels.long())
        val_losses.append(val_loss.item())
    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Val Loss: {sum(val_losses)/len(val_losses)}')

Epoch 1, Loss: 0.4848940372467041, Val Loss: 0.4547721489270528
Epoch 2, Loss: 0.3824653923511505, Val Loss: 0.45498765468597413
Epoch 3, Loss: 0.3853233754634857, Val Loss: 0.45382894555727643
Epoch 4, Loss: 0.4513022005558014, Val Loss: 0.4526953168710073
Epoch 5, Loss: 0.4542456865310669, Val Loss: 0.4548202689488729
Epoch 6, Loss: 0.4479007422924042, Val Loss: 0.4539025729894638
Epoch 7, Loss: 0.32521116733551025, Val Loss: 0.45341663678487143
Epoch 8, Loss: 0.48235607147216797, Val Loss: 0.4504246719678243
Epoch 9, Loss: 0.26368284225463867, Val Loss: 0.452763177951177
Epoch 10, Loss: 0.4475571811199188, Val Loss: 0.44804271121819816
Epoch 11, Loss: 0.38624855875968933, Val Loss: 0.448045175075531
Epoch 12, Loss: 0.3779705762863159, Val Loss: 0.44833411594231926
Epoch 13, Loss: 0.3736109435558319, Val Loss: 0.44657398025194806
Epoch 14, Loss: 0.431583970785141, Val Loss: 0.4461916430791219
Epoch 15, Loss: 0.4304144084453583, Val Loss: 0.4504768451054891
Epoch 16, Loss: 0.444591313

Testare model

In [38]:
model.eval()

correct = 0
total = 0
predictions = []
labelsTest = []
for inputs, labels in test_loader:
    with torch.no_grad():
        test_outputs = model(inputs.float())
        _, predicted = torch.max(test_outputs, 1)
        total += labels.size(0)
        predictions.extend(predicted)
        labelsTest.extend(labels.long())
        correct += (predicted == labels.long()).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy}')
print(classification_report(labelsTest, predictions))

Test Accuracy: 0.8305084745762712
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      3087
           1       0.50      0.00      0.00       630

    accuracy                           0.83      3717
   macro avg       0.67      0.50      0.46      3717
weighted avg       0.77      0.83      0.75      3717

